# Приветствую в DFL-Colab!

Это адаптированная версия DFL для Google Colab.


# Обзор
*   Экстрактор работает в полном функционале.
*   Обучение может работать без предварительного просмотра.
*   Объединение работает в полном функционале.
*   Вы можете импортировать/экспортировать рабочую область с помощью Google Диска.
*   Импорт/экспорт и другие манипуляции с рабочей областью вы можете выполнить в блоке «Управление рабочей областью».
*   Машина Google Colab активна в течение 12 часов. DFL-Colab делает резервную копию вашего рабочего пространства в режиме обучения.
*   Google не любит длительных тяжелых расчетов. Поэтому для обучения более двух сессий подряд используйте два аккаунта Google. Рекомендуется разделить обучение на две учетные записи, но вы можете использовать одну учетную запись Google Диска для хранения своего рабочего пространства.



## Предотвращение случайных отключений

Эта ячейка запускает код JS для автоматического повторного подключения к среде выполнения.

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Нажмите кнопку подключения к совместной работе"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Нажмите чтобы переподключиться"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Готово.")

## Проверка GPU

*   Google Colab может предоставить вам одну из видеокарт Tesla: K80, T4, P4 or P100
*   Здесь вы можете проверить модель графического процессора перед использованием DeepFaceLab.



In [ ]:
!nvidia-smi

## Установка или обновление DeepFaceLab

* Установите или обновите DeepFaceLab прямо с Github.
* Модули устанавливаются автоматически
* Автоматически устанавливает таймер для предотвращения случайных отключений.
* Опция «Download_FFHQ» означает загрузку высококачественного набора данных FFHQ вместо CelebA. FFHQ занимает больше памяти, поэтому его загрузка займет больше времени, чем CelebA. Рекомендуется включить эту опцию, если вы выполняете предварительную тренировку.

In [ ]:
#@title Установка или обновление DeepFaceLab с Github
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets
import IPython
from pathlib import Path

Mode = "install" #@param ["install", "update"]
Download_FFHQ = False #@param {type:"boolean"}

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

pretrain_link = "https://github.com/chervonij/DFL-Colab/releases/download/"
pretrain_link = pretrain_link+"pretrain_GenericFFHQ/pretrain_FFHQ.zip" if Download_FFHQ else pretrain_link+"pretrain-CelebA/pretrain_CelebA.zip"

if (Mode == "install"):
  !git clone https://github.com/iperov/DeepFaceLab.git
  %cd "/content/DeepFaceLab/"
  %cd "/content/"
else:
  %cd /content/DeepFaceLab/
  !git pull
clear_output()

with capture.capture_output() as cap:
  %pip install h5py==3.8.0
  %pip install opencv-python==4.7.0.72
  %pip install ffmpeg-python==0.2.0
  %pip install colorama==0.4.6
  %pip install tf2onnx==1.14.0
clear_output()

if not Path("/content/pretrain").exists():
  print("Downloading Pretrain faceset ... ")
  !wget -q --no-check-certificate -r $pretrain_link -O /content/pretrain_faceset.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_faceset.zip -d /content/pretrain/
  !rm /content/pretrain_faceset.zip
clear_output()

if not Path("/content/pretrain_Q96").exists():
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
  !mkdir /content/pretrain_Q96
  !unzip -q /content/pretrain_Q96.zip -d /content/pretrain_Q96/
  !rm /content/pretrain_Q96.zip
clear_output()

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  
clear_output()

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Нажмите кнопку подключения к совместной работе"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Нажмите для переподключения"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()
inf('\u2714 Готово','success', '50px')

## Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [ ]:
#@title Импорт с гугл диска.
from IPython.display import clear_output
import ipywidgets as widgets
from google.colab import drive

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

drive.mount('/content/drive')

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name
clear_output()

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
clear_output()
inf('\u2714 Готово','success', '50px')

In [ ]:
#@title Экспорт на диск { form-width: "30%" }
from IPython.display import clear_output
import ipywidgets as widgets

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "merged_mask", "models", "result video", "result_mask video"]
Archive_name = "workspace.zip" #@param {type:"string"}
drive.mount('/content/drive')

def zip_and_copy(path, mode):
  zip_cmd="-0 -r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name
clear_output()

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "merged_mask":
  zip_and_copy("/content/workspace/data_dst", "merged_mask")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
elif Mode == "result video":
  !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
  
clear_output()
inf('\u2714 Готово','success', '50px')

In [ ]:
#@title Импорт с URL{ form-width: "30%", display-mode: "form" }
from IPython.display import clear_output
import ipywidgets as widgets
import urllib.request
from pathlib import Path

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

URL = "http://" #@param {type:"string"}
Mode = "unzip to content" #@param ["unzip to content", "unzip to content/workspace", "unzip to content/workspace/data_src", "unzip to content/workspace/data_src/aligned", "unzip to content/workspace/data_dst", "unzip to content/workspace/data_dst/aligned", "unzip to content/workspace/model", "download to content/workspace"]

def unzip(zip_path, dest_path):

    
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd  
  rm_cmd = "rm "+dest_path + url_path.name
  !$rm_cmd
  clear_output()
  print("Разархивировано!")

if Mode == "unzip to content":
  dest_path = "/content/"
elif Mode == "unzip to content/workspace":
  dest_path = "/content/workspace/"
elif Mode == "unzip to content/workspace/data_src":
  dest_path = "/content/workspace/data_src/"
elif Mode == "unzip to content/workspace/data_src/aligned":
  dest_path = "/content/workspace/data_src/aligned/"
elif Mode == "unzip to content/workspace/data_dst":
  dest_path = "/content/workspace/data_dst/"
elif Mode == "unzip to content/workspace/data_dst/aligned":
  dest_path = "/content/workspace/data_dst/aligned/"
elif Mode == "unzip to content/workspace/model":
  dest_path = "/content/workspace/model/"
elif Mode == "download to content/workspace":
  dest_path = "/content/workspace/"
clear_output()

if not Path("/content/workspace").exists():
  cmd = "mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"
  !$cmd
clear_output()

url_path = Path(URL)
urllib.request.urlretrieve ( URL, dest_path + url_path.name )

if (url_path.suffix == ".zip") and (Mode!="download to content/workspace"):
  unzip(dest_path + url_path.name, dest_path)
  
clear_output()
inf('\u2714 Готово','success', '50px')

In [ ]:
#@title Экспорт с URL
from IPython.display import clear_output
import ipywidgets as widgets

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

URL = "http://" #@param {type:"string"}
Mode = "upload workspace" #@param ["upload workspace", "upload data_src", "upload data_dst", "upload data_src aligned", "upload data_dst aligned", "upload merged", "upload model", "upload result video"]

cmd_zip = "zip -0 -r -q "

def run_cmd(zip_path, curl_url):
  cmd_zip = "zip -0 -r -q "+zip_path
  cmd_curl = "curl --silent -F "+curl_url+" -D out.txt > /dev/null"
  !$cmd_zip
  !$cmd_curl
clear_output()

if Mode == "upload workspace":
  %cd "/content"
  run_cmd("workspace.zip workspace/","'data=@/content/workspace.zip' "+URL)
elif Mode == "upload data_src":
  %cd "/content/workspace"
  run_cmd("data_src.zip data_src/", "'data=@/content/workspace/data_src.zip' "+URL)
elif Mode == "upload data_dst":
  %cd "/content/workspace"
  run_cmd("data_dst.zip data_dst/", "'data=@/content/workspace/data_dst.zip' "+URL)
elif Mode == "upload data_src aligned":
  %cd "/content/workspace"
  run_cmd("data_src_aligned.zip data_src/aligned", "'data=@/content/workspace/data_src_aligned.zip' "+URL )
elif Mode == "upload data_dst aligned":
  %cd "/content/workspace"
  run_cmd("data_dst_aligned.zip data_dst/aligned/", "'data=@/content/workspace/data_dst_aligned.zip' "+URL)
elif Mode == "upload merged":
  %cd "/content/workspace/data_dst"
  run_cmd("merged.zip merged/","'data=@/content/workspace/data_dst/merged.zip' "+URL )
elif Mode == "upload model":
  %cd "/content/workspace"
  run_cmd("model.zip model/", "'data=@/content/workspace/model.zip' "+URL)
elif Mode == "upload result video":
  %cd "/content/workspace"
  run_cmd("result.zip result.mp4", "'data=@/content/workspace/result.zip' "+URL)
  
  
!rm *.zip

%cd "/content"
clear_output()
inf('\u2714 Готово','success', '50px')

In [ ]:
#@title Удаление и создание заново
from IPython.display import clear_output
import ipywidgets as widgets

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged; rm -r /content/workspace/data_dst/merged_mask"
  
!$cmd
clear_output()
inf('\u2714 Готово','success', '50px')

## Инструменты извлечения, сортировки и настройки граней
* Извлечение кадров для видео SRC или DST.
* Видео Denoise SRC или DST. Параметр «Фактор» устанавливает интенсивность шумоподавления.
* Обнаружение и выравнивание лиц. При необходимости можно получить кадры с отладочными ориентирами.
* Экспортируйте рабочую область на Google Диск после извлечения и сортировки вручную (в блоке «Управление рабочей областью»)
* Вы можете улучшить свои наборы лиц с помощью DFL FacesetEnhancer.
* Измените размер грани в соответствии с разрешением вашей модели. Поскольку у Colab нет мощного процессора, изменение размера выборки во время обучения увеличивает время итерации. Изменение размера Faceset сокращает время итерации примерно в 2 раза. Не забудьте сохранить исходный Faceset на своем компьютере.
* Упакуйте или распакуйте наборы граней с помощью инструмента упаковки DFL.
* Примените или удалите обученную модель XSeg к извлеченным граням.
*Рекомендуется к использованию, универсальная модель XSeg для автоматической сегментации.


In [ ]:
#@title Работа с кадрами.
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [ ]:
#@title Кадры с шумоподавлением
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [ ]:
#@title Обнаружение лиц
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (whole face)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "whole face" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content"
!python $cmd

In [ ]:
#@title Сортировать по выравниванию
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "motion-blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final-by-blur", "final-by-size", "absdiff"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [ ]:
#@title Усилитель Faceset
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/workspace/"+Data+"/aligned"
cmd = "/content/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [ ]:
#@title Изменение размера грани
Data = "data_src" #@param ["data_src", "data_dst"]

cmd = "/content/DeepFaceLab/main.py facesettool resize --input-dir /content/workspace/" + \
      f"{Data}/aligned"

!python $cmd

In [ ]:
#@title Упаковать/распаковать выровненный набор граней

Folder = "data_src" #@param ["data_src", "data_dst"]
Mode = "unpack" #@param ["pack", "unpack"]

cmd = "/content/DeepFaceLab/main.py util --input-dir /content/workspace/" + \
      f"{Folder}/aligned --{Mode}-faceset"

!python $cmd

In [ ]:
#@title Применить или удалить маску XSeg к граням
Mode = "Apply mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]
GenericXSeg = True #@param {type:"boolean"}

from pathlib import Path
mode_arg = 'apply' if Mode == "Apply mask" else 'remove'

if GenericXSeg and not Path('/content/GenericXSeg').exists():
  print('Downloading Generic XSeg model ... ')
  xseg_link = 'https://github.com/chervonij/DFL-Colab/releases/download/GenericXSeg/GenericXSeg.zip'
  !mkdir /content/GenericXSeg
  !wget -q --no-check-certificate -r $xseg_link -O /content/GenericXSeg.zip
  !unzip -q /content/GenericXSeg.zip -d /content/GenericXSeg/
  !rm /content/GenericXSeg.zip

main_path = '/content/DeepFaceLab/main.py'
data_path = f'/content/workspace/{Data}/aligned'
model_path = '/content/workspace/model' if not GenericXSeg else '/content/GenericXSeg'

cmd = f'{main_path} xseg {mode_arg} --input-dir {data_path} '
cmd += f'--model-dir {model_path}' if mode_arg == 'apply' else ''

!python $cmd

## Тренировка модели

* Выберите тип модели, но SAEHD рекомендуется всем.
* Установите параметры модели в поле вывода.
* Вы можете просмотреть предварительный просмотр вручную, если зайдете в папку модели в файловом менеджере и дважды щелкните файл Preview.jpg.
* Ваше рабочее пространство будет заархивировано и загружено на подключенный диск через 11 часов с момента начала сеанса.
* Если вы выберете опцию «Backup_every_hour», резервное копирование вашего рабочего пространства будет выполняться каждый час.
* Также вы можете экспортировать свое рабочее пространство вручную в блоке «Управление рабочим пространством».
* Опция «Silent_Start» обеспечивает автоматический запуск с лучшим графическим процессором и последней использованной моделью.

In [ ]:
#@title Тренировка
import psutil, os, time
from google.colab import drive
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96", "XSeg"]
Backup_every_hour = True #@param {type:"boolean"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content"
drive.mount('/content/drive')

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -0 -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(3600)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -0 -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -0 -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

## Объединить кадры

In [ ]:
#@title Объединить кадры
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96" ]

cmd = "DeepFaceLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Получить видео результата
Mode = "result video" #@param ["result video", "result_mask video"]
Copy_to_Drive = True #@param {type:"boolean"}


if Mode == "result video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio
  if Copy_to_Drive:
    !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4
  if Copy_to_Drive:
    !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
